In [3]:
import torch

w = torch.tensor([2.0, 3.0], requires_grad=True)

output = w[0] * w[1]

w += 1.0  

output.backward()

print(w.grad)

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

In [ ]:
w = torch.tensor([2.0, 3.0], requires_grad=True)

output = w[0] * w[1]

w = w + 1.0

output.backward()

print(w.grad)

None


C:\Users\kingv\AppData\Local\Temp\ipykernel_6632\209522001.py:11: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:494.)
  print(w.grad)


In [7]:
w = torch.tensor([2.0, 3.0], requires_grad=True)

output = w[0] * w[1]

output.backward()
print("Gradient:", w.grad)  

with torch.no_grad():
    w += 1.0          

print("Updated w:", w)

Gradient: tensor([3., 2.])
Updated w: tensor([3., 4.], requires_grad=True)
